# Warning

This notebook uses instaloader, a powerful but dangerous instagram scraping library.  
Only use it with secondary accounts! There is a high risk of temporary or permanent bans.  
It is very useful for some initial or small data batches (maybe a post or so), but after that it gets dangerous.

A user on the instaloader GitHub had a good description here:
 > MHO this is a very important topic. Sharing my observation from 18 months or so using Instaloader's Python module.  
I noticed that to avoid triggering Instagram's bad behavior detectors, besides Instaloader's code behaving as similarly as possible to an Instagram client
(thank you repo maintainers), we users must be aware of, and avoid, behaviors that can trigger accounts and/or IP address blocks.

> Bad behavior detectors would be triggered even if it was Instagram's own app/webapp. For example, if you try to access some IG post URLs with anonymous sessions from a browser multiple times in a row, Instagram will route you to the login page. Keep doing it and it'll temporarily block your IP address. Keep insisting even more, and it may flag your IP as authenticated session only, no anonymous access. Keep doing it over and over, even with an authenticated session it'll flag your account, temporarily or for good. It's happened to my home IP address and many test accounts even before trying Instaloader.

> So, I've been able to reasonably to use Instaloader at scale by:
a) Authenticating accounts via Firefox with some automation via playright to collect the cookies;
b) Using a rotating residential proxy that allow multiple requests per session to scatter Instaloader sessions over multiple IP addresses and geographies (single request proxy connections won't work);
c) Limiting the rate of requests per account in authenticated sessions. Yes, it requires some effort to produce and take care of more and mores accounts if you need to frequently collect lots of stories, but it is what it is.

[Source](https://github.com/instaloader/instaloader/issues/1984#issuecomment-1696375143)

Also, to get instaloader to authenticate correctly, you need to modify the library files itself (at least in version 4.14).

Please go to the instaloadercontext.py file and remove `'sessionid': '',` from the `session.cookies.update()`method call dictionary in the `login()` method.
You also need to either use direct login via username and password or use a cookie from the browser directly to log in. Sessions are not working correctly as of now.

[Source](https://github.com/instaloader/instaloader/issues/2487#issue-2807621924)

In [ ]:
from instaloader import Instaloader, Profile
from my_secrets import INSTA_ACCOUNT_USERNAME, INSTA_ACCOUNT_PASSWORD 
L = Instaloader(user_agent="chrome")
L.login(INSTA_ACCOUNT_USERNAME, INSTA_ACCOUNT_PASSWORD)

In [ ]:
profile = Profile.from_username(L.context, "dorfterror")
profile.username, profile.full_name, profile.biography, profile.external_url, profile.followers, profile.followees, profile.is_verified

# Attention

The following code was just for experimenation and data exploration.  
Running the whole post scraping for loop on a profile will very likely get your banned.

### BE CAREFUL

In [ ]:
import sqlite3

conn = sqlite3.connect('instagram_posts.db')
c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        shortcode TEXT PRIMARY KEY,
        caption TEXT,
        likes INTEGER,
        comments_count INTEGER,
        comments TEXT,
        commenters TEXT,
        co_authors TEXT
    )
''')
conn.commit()



for i, post in enumerate(profile.get_posts()):
    print(f"Processing post: {post.shortcode}")
    shortcode = post.shortcode
    caption = post.caption
    likes = post.likes
    comments_count = post.comments
    print(f"Likes: {likes}, Comments: {comments_count}")

    comments_list = []
    commenters_list = []

    for comment in post.get_comments():
        comments_list.append(comment.text)
        commenters_list.append(comment.owner.username)

    c.execute('''
        INSERT OR REPLACE INTO posts (shortcode, caption, likes, comments_count, comments, commenters)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        shortcode,
        caption,
        likes,
        comments_count,
        "\n".join(comments_list),
        ", ".join(commenters_list),
    ))

    conn.commit()
    if i == 5:  # Limit to first 5 posts for testing
        break

conn.close()


Processing post: DBavDJ7MpfB
Likes: 516, Comments: 14
Processing post: DCWBEZRM1TC
Likes: 221, Comments: 26
Processing post: DLxrfqfMSvp
Likes: 158, Comments: 2
Processing post: DLuXtgJs9W0
Likes: 114, Comments: 3
Processing post: DLj1jdAMTnt
Likes: 48, Comments: 0
Processing post: DLaONVUIGwb
Likes: 55, Comments: 3


In [15]:
print(post._owner_profile)

<Profile riez_openair (7160047809)>


In [12]:
temp = post._iphone_struct
print(temp)

{'code': 'DLaONVUIGwb', 'pk': '3664303735709920283', 'id': '3664303735709920283_7160047809', 'ad_id': None, 'boosted_status': None, 'boost_unavailable_identifier': None, 'boost_unavailable_reason': None, 'caption': {'has_translation': True, 'created_at': 1751040009, 'pk': '18090292417629160', 'text': 'Gute vier Wochen sind es noch bis zu eurer Lieblingsgartenparty in der Jubiläumsedition und am Freitag werdet ihr dort auf @dorfterror treffen. Hört euch an was Fussel zu sagen hat und checkt sie auf den gängigen Kanälen aus falls ihr das noch nicht getan habt, es lohnt sich. Riez Open Air 2025: 24. - 26.07.2025. Ticketshop, Hörprobe, Infos findet ihr unter https://linktr.ee/riezopenair #roa2025 #riezopenair festivalgartenparty punkrock hardcore #itchy #montreal #jupiterjones #bane #crystallake #captainplanet #themeffs #timvantol #dunerats #knochenfabrik #desaster #bubonix #thepill #waronwomen #theironroses #phantombay #indecentbehavior #christmas #theredflags #statepower #fuckingangry #d

In [ ]:
from pprint import pprint
pprint(post._asdict())

{'__isXDTGraphMediaInterface': 'XDTGraphVideo',
 '__typename': 'GraphVideo',
 'accessibility_caption': None,
 'can_see_insights_as_brand': False,
 'caption': 'Gute vier Wochen sind es noch bis zu eurer Lieblingsgartenparty '
            'in der Jubiläumsedition und am Freitag werdet ihr dort auf '
            '@dorfterror treffen. Hört euch an was Fussel zu sagen hat und '
            'checkt sie auf den gängigen Kanälen aus falls ihr das noch nicht '
            'getan habt, es lohnt sich. Riez Open Air 2025: 24. - 26.07.2025. '
            'Ticketshop, Hörprobe, Infos findet ihr unter '
            'https://linktr.ee/riezopenair #roa2025 #riezopenair '
            'festivalgartenparty punkrock hardcore #itchy #montreal '
            '#jupiterjones #bane #crystallake #captainplanet #themeffs '
            '#timvantol #dunerats #knochenfabrik #desaster #bubonix #thepill '
            '#waronwomen #theironroses #phantombay #indecentbehavior '
            '#christmas #theredflags #statep

In [ ]:
followers = profile.get_followers()

data = []
for follower in followers:
    data.append({
        "id": follower.userid,
        "username": follower.username,
        "full_name": follower.full_name,
        "follower_count": follower.followers,
        "post_count": follower.mediacount
    })
    if len(data) >= 10:  # Limit to first 10 followers for testing
        break
print(data)

In [ ]:
from pprint import pprint
pprint(follower._asdict())

{'ai_agent_type': None,
 'bio_links': [],
 'biography': 'life enthusiast and fellow musician\n'
              'singing in @bulletinsideband band',
 'biography_with_entities': {'entities': [{'hashtag': None,
                                           'user': {'username': 'bulletinsideband'}}],
                             'raw_text': 'life enthusiast and fellow musician\n'
                                         'singing in @bulletinsideband band'},
 'blocked_by_viewer': False,
 'business_address_json': None,
 'business_category_name': None,
 'business_contact_method': 'UNKNOWN',
 'business_email': None,
 'business_phone_number': None,
 'category_enum': None,
 'category_name': 'Musician/band',
 'country_block': False,
 'edge_follow': {'count': 1292},
 'edge_followed_by': {'count': 359},
 'edge_mutual_followed_by': {'count': 0, 'edges': []},
 'eimu_id': '17843885537184327',
 'external_url': None,
 'external_url_linkshimmed': None,
 'fb_profile_biolink': None,
 'fbid': '17841435529801973

In [ ]:
comments = post.get_comments()
print(comments[0].answers)

## Scraping with Residential Proxy

In [2]:
from utils import add_project_root_to_path
add_project_root_to_path()

In [2]:
import random
import time
import pandas as pd
from instaloader import Instaloader, Post as InstaPost 
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from dorfterror_database.database_structure import Post as DBPost
from pathlib import Path
import requests
from scraping_notebooks.my_secrets import INSTA_USER_2, INSTA_PWD_2

USERNAME = INSTA_USER_2
PASSWORD = INSTA_PWD_2

root_path = Path(".").resolve()
engine = create_engine(f"sqlite:///{root_path}/dorfterror_database/dorfterror.db")
Session = sessionmaker(bind=engine)
session = Session()

L = Instaloader()
session_req = requests.Session()
try:
    L.login(USERNAME, PASSWORD)
    print("✅ Login erfolgreich.")
except Exception as e:
    print(f"❌ Login fehlgeschlagen: {e}")
    raise SystemExit("Beende das Skript nach fehlgeschlagenem Login.")

already_queried_posts = pd.read_csv("all_post_likes.csv")
db_posts = session.query(DBPost).filter(DBPost.shortcode != None).filter(~DBPost.id.in_(already_queried_posts["post_id"].unique().tolist())).all()

all_likes_df = already_queried_posts.copy()
for idx, db_post in enumerate(db_posts):
    shortcode = db_post.shortcode
    print(f"🔍 Lade Likes für Post: {shortcode} ({idx + 1}/{len(db_posts)})")
    like_data = []
    failures = 0
    try:
        post = InstaPost.from_shortcode(L.context, shortcode)
        current_likes = post.get_likes()
        print(f"Likes: {post.likes}")
        for liker in current_likes:
            like_data.append({
                "post_id": db_post.id,
                "username": liker.username,
                "user_id": liker.userid
            })

        time.sleep(random.uniform(5, 12))

    except Exception as e:
        print(f"⚠️ Fehler bei {shortcode}: {e}")
        failures += 1
        if failures >= 3:
            print(f"❌ Zu viele Fehler bei {shortcode}. Beende das Skript.")
            raise SystemExit("Beende das Skript nach zu vielen Fehlern.")
        else:
            print(f"⏳ Wartezeit vor erneutem Versuch bei {shortcode}...")
            time.sleep(random.uniform(150, 250))

    likes_df = pd.DataFrame(like_data)
    all_likes_df = pd.concat([all_likes_df, likes_df], ignore_index=True)
    all_likes_df.to_csv("all_post_likes.csv", index=False)
    print(f"✅ Likes für {shortcode} gespeichert. Aktuell: {len(all_likes_df)} Likes insgesamt.")

    if idx % 10 == 0:
        print(f"Längere Pause nach {idx} Posts...")
        time.sleep(150)

print(f"✅ Fertig! {len(all_likes_df)} Likes gespeichert.")


✅ Login erfolgreich.
🔍 Lade Likes für Post: DHHE-uTOM7N (1/478)
Likes: -1
✅ Likes für DHHE-uTOM7N gespeichert. Aktuell: 11009 Likes insgesamt.
Längere Pause nach 0 Posts...
🔍 Lade Likes für Post: DHBjRYoMKJU (2/478)
Likes: -1
✅ Likes für DHBjRYoMKJU gespeichert. Aktuell: 11210 Likes insgesamt.
🔍 Lade Likes für Post: DG8xLJxthg- (3/478)
Likes: -1
✅ Likes für DG8xLJxthg- gespeichert. Aktuell: 11401 Likes insgesamt.
🔍 Lade Likes für Post: DG0z7NjMMnR (4/478)
Likes: -1
✅ Likes für DG0z7NjMMnR gespeichert. Aktuell: 11462 Likes insgesamt.
🔍 Lade Likes für Post: DGxOWx1ssSq (5/478)
Likes: -1
✅ Likes für DGxOWx1ssSq gespeichert. Aktuell: 11556 Likes insgesamt.
🔍 Lade Likes für Post: DGk-6vZM_zU (6/478)
Likes: -1
✅ Likes für DGk-6vZM_zU gespeichert. Aktuell: 11633 Likes insgesamt.
🔍 Lade Likes für Post: DGitY7DMa0B (7/478)
Likes: -1
✅ Likes für DGitY7DMa0B gespeichert. Aktuell: 11745 Likes insgesamt.
🔍 Lade Likes für Post: DGdIv7JtJUC (8/478)
Likes: -1
✅ Likes für DGdIv7JtJUC gespeichert. Aktue

## Hashtag Scraper

In [1]:
from utils import add_project_root_to_path
add_project_root_to_path()

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dorfterror_database.database_structure import Post
from pathlib import Path
import json

root_path = Path(".").resolve()
engine = create_engine(f"sqlite:///{root_path}/dorfterror_database/dorfterror.db")
Session = sessionmaker(bind=engine)
session = Session()

# Alle Hashtags aus der Datenbank extrahieren
all_hashtags = set()
posts = session.query(Post.caption_hashtags).filter(Post.caption_hashtags.isnot(None)).all()

for (hashtags,) in posts:
    if isinstance(hashtags, list):
        all_hashtags.update(hashtags)
    elif isinstance(hashtags, str):
        try:
            all_hashtags.update(json.loads(hashtags))
        except json.JSONDecodeError:
            continue

all_hashtags = sorted(all_hashtags)
print(f"Gesammelte Hashtags: {len(all_hashtags)}")
print(all_hashtags[:10])


Gesammelte Hashtags: 961
['#161', '#161crew', '#16days', '#1805', '#1k', '#2', '#2019', '#2021', '#2022', '#2023']


In [4]:
import time
import random
import pandas as pd
from instaloader import Instaloader, Hashtag
import requests
from scraping_notebooks.my_secrets import DOMINIK_USER as USERNAME, DOMINIK_PWD as PASSWORD

L = Instaloader()
session_req = requests.Session()
try:
    L.login(USERNAME, PASSWORD)
    print("✅ Login erfolgreich.")
except Exception as e:
    print(f"❌ Login fehlgeschlagen: {e}")
    raise SystemExit("Beende das Skript nach fehlgeschlagenem Login.")

try:
    already_scraped_hashtags = pd.read_csv("already_scraped_hashtags.csv")
except FileNotFoundError:
    already_scraped_hashtags = pd.DataFrame(columns=["hashtag", "mediacount"])

already_scraped_hashtags = already_scraped_hashtags.dropna(subset=["hashtag", "mediacount"])

filtered_hashtags = set(all_hashtags)-set(already_scraped_hashtags["hashtag"].tolist())

for idx, hashtag  in enumerate(filtered_hashtags):
    failed = False
    print(f"🔍 Verarbeite Hashtag: {hashtag} ({idx + 1}/{len(filtered_hashtags)})")
    try:
        current_hashtag = Hashtag.from_name(L.context, hashtag.lstrip("#"))
        mediacount = current_hashtag.mediacount
        current_results = pd.DataFrame([{"hashtag": hashtag, "mediacount": mediacount}])
    except Exception as e:
        failed = True
        print(f"Fehler bei {hashtag}: {e}")
        current_results = pd.DataFrame([{"hashtag": hashtag, "mediacount": None}])
    
    already_scraped_hashtags = pd.concat([already_scraped_hashtags, current_results], ignore_index=True)
    already_scraped_hashtags.to_csv("already_scraped_hashtags.csv", index=False)
    if failed:
        print(f"❌ Fehler bei {hashtag}. Überspringe...")
    else:
        print(f"✅ {hashtag} verarbeitet. Mediacount: {mediacount}.")
    time.sleep(random.uniform(30, 45))
    
    if (idx + 1) % 25 == 0:
        print(f"{idx+1} Hashtags verarbeitet – kurze Pause...")
        time.sleep(150)

print(f"✅ Fertig! {len(filtered_hashtags)} Hashtags gespeichert.")


✅ Login erfolgreich.
🔍 Verarbeite Hashtag: #götzwidmann (1/567)
Fehler bei #götzwidmann: 400 Bad Request - "fail" status, message "feedback_required" when accessing https://i.instagram.com/api/v1/tags/web_info/?__a=1&__d=dis&tag_name=g%C3%B6tzwidmann
❌ Fehler bei #götzwidmann. Überspringe...


/tmp/ipykernel_771/3378602038.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  already_scraped_hashtags = pd.concat([already_scraped_hashtags, current_results], ignore_index=True)


KeyboardInterrupt: 

In [4]:
import pandas as pd
already_scraped_hashtags = pd.read_csv("already_scraped_hashtags.csv")
display(already_scraped_hashtags.isna().sum())

hashtag        0
mediacount    16
dtype: int64

## Collaborator Follower Scraper